# 爬虫简介

爬虫一般指从网站上获得需要的数据，这个过程其实是建造一个网站的逆向过程。

# 使用Flask建立简单网站

所谓知己知彼，则百战不殆。既然爬虫主要是从网站上爬取数据，那么只有对建造网站的步骤一清二楚，才能在变幻多端的网站环境下，获取想要的数据。

然而实际建立网站的过程是非常繁琐而复杂的，写HTML、CSS仅仅是一个网站的冰山一角，这其中至少涉及到网站架构设计、数据库设计、后台服务设计、前端设计等等，此外根据不同的架构设计，还有很多中间件、缓存数据库等等复杂的设计。

不过，对于比较简单的网站，Python本身就有很多成熟的框架可以方便我们快速搭建一个简单的网站。这其中，Flask由于其比较精美的设计架构以及简单的模板等应用，是非常受欢迎的轻量级Web框架。在这里，我们不妨使用Flask搭建一个最简单的网站（因为在Jupyter里面，所以我把run()给注释掉了，如果需要执行，请直接执行html/web.py）：
```python
from flask import Flask
app = Flask(__name__)
import random

@app.route('/')
def index():
    with open("example4.html") as f:
        html=f.read()
    return html

@app.route('/dynamic')
def dynamic():
    with open("dynamic.html") as f:
        html=f.read()
    return html

@app.route('/dynamic_response')
def dynamic_response():
    return str(random.random())

if __name__ == '__main__':
    #app.run()
    pass
```

在执行以上命令之前，需要首先使用pip install flask安装Flask。

在上面的程序中，@代表修饰器（decorator），是Python编程的一个高级特性，我们暂且不管。我们需要知道的仅仅是，通过@app.route函数，声明了一个路径，该路径即访问接下来定义的页面的路径。

在@app.route下方，我们定义了几个函数，这几个函数的作用是返回一个字符串，这些字符串会通过网络传递给访问的浏览器。

如果运行html/web.py，会提示：
>  Running on http://127.0.0.1:5000/

此时，如果在浏览器中输入以上网址，服务器就会执行index()函数，该函数会读取example4.html，并将其返回，从而我们在浏览器上就看到了example4.html。

同理，如果访问 http://127.0.0.1:5000/dynamic ，服务器就会执行dynamic()函数，将dynamic.html的内容返回。

而如果访问 http://127.0.0.1:5000/dynamic_response ，服务器会执行dynamic_response()函数，该函数生成一个随机数并返回给浏览器。

以上就是一个简单的网站。

# 发送请求

知道了服务器端如何处理页面，那么现在的问题是，客户端怎样与服务器端通讯呢？

这里就要介绍以下HTTP协议的概念了。

HTTP协议是**Hyper Text Transfer Protocol**（**超文本传输协议**）的缩写，一种服务器/客户端范式的传输协议，我们访问网址（URL）时，都是以http:// 开头的，或者https:// 开头，代表的就是使用http协议。

一个最简单的传输模型是，客户端通过URL地址向服务器端发送**请求**（**request**），服务器根据所请求的地址、头部信息（headers）做出**响应**（**response**）。

这里有几个概念要特别注意：

## 网址URL

一个常见的网址通常具有如下形式：

> http://econpaper.cn:8080/article/article.jsp?id=56987&userid=3455

其中：

* http:// 代表协议
* econpaper.cn 代表域名
* :8080 代表端口号，默认为80端口
* /article/article.jsp 部分为请求的页面路径
* ?id=56987&userid=3455 为需要传递给这个页面的参数

## 请求（request）

客户端向服务器端发送请求，要按照一定的格式，请求消息由以下三部分组成：

* 请求行（request line）
* 头部（header）
* 请求数据

![](pic/request.png)

比如，以下是一个典型的请求头部：

![](pic/request_headers.png)

这些信息传递给服务器后，服务器根据这些信息进行处理。

实际上，在我们刚刚运行的Flask中，在客户端可以看到每一次的请求。

此外，上面从URL地址中已经看到，?id=56987&userid=3455 为需要传递给这个页面的参数，实际上，这是一个GET的请求方法。为了从客户端向服务器端传数据，以下两种方法是最常用的：

* GET：像上面一样，请求的数据明文写在URL上
* POST：数据包含在请求体中

当然，两种方法也可以结合起来使用。不过最为常用的仍然是GET方法。

## 响应（response）

有了请求，就会有服务器的响应。同样，响应也有响应头和数据体。在响应头中，最重要的是响应的状态码以及内容类型（html文档或者图片、pdf等），常见的状态码比如：

* 200 OK                        客户端请求成功
* 400 Bad Request               客户端请求有语法错误，不能被服务器所理解
* 401 Unauthorized              请求未经授权，这个状态代码必须和WWW-Authenticate报头域一起使用 
* 403 Forbidden                 服务器收到请求，但是拒绝提供服务
* 404 Not Found                 请求资源不存在，eg：输入了错误的URL
* 500 Internal Server Error     服务器发生不可预期的错误
* 503 Server Unavailable        服务器当前不能处理客户端的请求，一段

# Python中发送请求

在Python中，有很多工具可以帮助我们向服务器发送请求，包括但不限于：urllib、urllib2、urllib3、httplib2、requests、httpx等等。出于个人偏好原因，在这里我们以requests为例，介绍如何发送请求。此外，如果需要使用Python中的异步特性，可以使用httpx，其使用方法与requests类似，我们在这不再赘述。当然，前提是需要熟练掌握async、await等异步编程的基本用法。

为了使用requests，需要先进行安装：pip install requests

使用时直接导入：
```python
import requests
```

接着，最简单的请求即GET请求：
```python
r=requests.get(url)
```

其中url为需要请求的地址。

有时我们在发送请求时可能需要控制发送请求的头部，头部可以使用一个字典表示，比如：
```python
headers={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36 OPR/60.0.3255.70",
        "Accept-Encoding":"gzip, deflate, lzma, sdch",
        "Accept-Language":"en-US,en;q=0.8",
        "Connection":"keep-alive",
        "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}
```

以上字典定义了头部的内容，接下来只要在get()函数中加入该头部就可以了：
```python
r=requests.get(url, headers=hearders)
```

返回结果：

* r.text 可以读取返回的HTML
* r.json() 获得Json数据转换后的字典数据
* r.encoding 记录了返回数据的字符编码
* r.status_code 记录了响应的状态码

如果需要发送POST请求，需要首先将请求的数据写成字典形式，在使用urllib包中的urlencode函数将其编码，比如：
```python
data=dict(name="Joe", comment="A test comment")
r=requests.post(url, data=data)
```

如果需要设置自定义的cookie到服务器，可以使用：
```python
cookies = dict(cookies_are='working')
r=requests.get(url, cookies=cookies)
```

如果我们希望设置最长的等待时间，可以使用超时选项：
```python
r=requests.get(url, timeout=10)
```

即设定等待时间超过10s则放弃连接。

最后，如果需要使用代理，可以使用：
```python
proxies = {
  'http': 'http://user@password@10.10.1.10:3128',
  'https': 'http://10.10.1.10:1080',
}
r=requests.get(url, proxies=proxies)
```

比如，联合国Comtrade数据库的API：

In [1]:
import requests

url = "http://comtrade.un.org/api/get?max=50000&type=C&freq=A&px=HS&ps=2013&r=826&p=0&rg=all&cc=AG2&fmt=json"

r = requests.get(url)
data = r.json()
data

{'validation': {'status': {'name': 'Ok',
   'value': 0,
   'category': 0,
   'description': '',
   'helpUrl': 'For more reference visit http://comtrade.un.org/data/dev/portal/'},
  'message': None,
  'count': {'value': 371,
   'started': '2022-04-06T16:32:55.8745631+02:00',
   'finished': '2022-04-06T16:32:56.0477594+02:00',
   'durationSeconds': 0.1731963},
  'datasetTimer': {'started': '2022-04-06T16:32:55.8745631+02:00',
   'finished': '2022-04-06T16:32:56.5321064+02:00',
   'durationSeconds': 0.6575432999999999}},
 'dataset': [{'pfCode': 'H4',
   'yr': 2013,
   'period': 2013,
   'periodDesc': '2013',
   'aggrLevel': 2,
   'IsLeaf': 0,
   'rgCode': 1,
   'rgDesc': 'Import',
   'rtCode': 826,
   'rtTitle': 'United Kingdom',
   'rt3ISO': 'GBR',
   'ptCode': 0,
   'ptTitle': 'World',
   'pt3ISO': 'WLD',
   'ptCode2': None,
   'ptTitle2': '',
   'pt3ISO2': '',
   'cstCode': '',
   'cstDesc': '',
   'motCode': '',
   'motDesc': '',
   'cmdCode': '01',
   'cmdDescE': 'Animals; live',
   

有的接口都需要用一个token进行身份验证，一般会根据token进行一定的数量限制。

以下代码实现了从链家上爬去房价数据。值得注意的是，如果没有设定headers，可能会爬不下来：

In [2]:
import requests
from bs4 import BeautifulSoup
import re

headers = {
    "User-Agent":
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36 OPR/60.0.3255.70",
    "Accept-Encoding":
    "gzip, deflate, lzma, sdch",
    "Accept-Language":
    "en-US,en;q=0.8",
    "Connection":
    "keep-alive",
    "Accept":
    "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

url = 'https://sh.lianjia.com/ershoufang/songjiang/'
r = requests.get(url, headers=headers)
html = r.text
bs = BeautifulSoup(html, "html.parser")
iterms = bs.find_all(name='div', attrs={"class": "info clear"})
for it in iterms:
    house_id = it.find(name='a', attrs={'data-housecode':
                                        re.compile('.+')})['data-housecode']
    position = it.find(name='div', attrs={'class': 'positionInfo'})
    position = list(position.find_all(name='a'))
    housing_estate = position[0].text
    location = position[1].text
    prop = it.find(name='div', attrs={'class': 'houseInfo'}).text
    ting_shi = re.search(r"\d室\d厅", prop).group()
    area = re.search(r"\d*(\.\d*)?平米", prop).group()
    price = it.find(name='div', attrs={'class': 'totalPrice totalPrice2'}).text
    follower = it.find(name='div', attrs={'class': 'followInfo'}).text
    print(
        [house_id, housing_estate, location, ting_shi, area, price, follower])


['107105087272', '珠江新城 ', '松江新城', '5室2厅', '253.48平米', ' 880万', '66人关注 / 16天以前发布']
['107105047788', '荣乐水岸 ', '松江老城', '3室2厅', '102.21平米', ' 363万', '17人关注 / 28天以前发布']
['107105055314', '紫薇茗庭 ', '泗泾', '3室1厅', '83.61平米', ' 285万', '36人关注 / 26天以前发布']
['107104722488', '珠江新城 ', '松江新城', '4室2厅', '137平米', ' 513万', '27人关注 / 4个月以前发布']
['107105047333', '通波小区 ', '松江老城', '3室2厅', '92.81平米', ' 356万', '13人关注 / 28天以前发布']
['107105108942', '月厦新天地 ', '松江老城', '1室1厅', '47.54平米', ' 114万', '25人关注 / 7天以前发布']
['107103708238', '海派青城 ', '新桥', '3室2厅', '126平米', ' 480万', '203人关注 / 一年前发布']
['107104159576', '新城上坤樾山明月 ', '泗泾', '3室2厅', '161.65平米', ' 1380万', '146人关注 / 9个月以前发布']
['107104784706', '茉莉雅苑 ', '泗泾', '2室1厅', '75.11平米', ' 249万', '179人关注 / 3个月以前发布']
['107104983400', '保利西子湾 ', '松江大学城', '2室2厅', '89.85平米', ' 490万', '35人关注 / 1个月以前发布']
['107104653131', '荣都公寓 ', '松江老城', '3室2厅', '120.54平米', ' 305万', '92人关注 / 4个月以前发布']
['107104490245', '华亭荣园 ', '松江老城', '3室2厅', '114.83平米', ' 425万', '19人关注 / 6个月以前发布']
['107104090174', '塘和家园德宁苑 '

# 使用selenium/playwright爬取动态页面

现在，很多网页通过使用JavaScript的Ajax技术实现了网页内容的动态改变。

比如，一个很简单的例子，如果访问我们之前建立的web.py中的网址： http://127.0.0.1:5000/dynamic 上面会有一个按钮，每按一次，该页面就会向服务器再发一个请求，服务器收到请求后会随机产生一个数字，并返回。所以每一次点击按钮，网页内容都会动态改变。

在碰到动态页面时，一种方法是精通JavaScript，并使用浏览器跟踪浏览器行为，分析JavaScript脚本，进而使用以上的方法模拟浏览器请求。但是这种方法非常复杂，很多时候JavaScript可能会复杂到一定程度，使得分析异常困难。

而另外一种方法，即使用selenium直接调用浏览器，浏览器自动执行JavaScript，然后调用浏览器的HTML即可。这种方法非常方便，但是速度异常之慢。

为了实现这一方法，我们首先要安装selenium：pip install selenium

除此之外，还要安装浏览器的支持。几种常见的浏览器支持插件的下载地址：

* firefox：https://github.com/mozilla/geckodriver/releases （对于Windows，下载后放到C:\Windows下；对于Linux/Mac，放到/usr/local/bin下。）
* chrome： https://sites.google.com/a/chromium.org/chromedriver/downloads
* safari： https://webkit.org/blog/6900/webdriver-support-in-safari-10/
* edge：   https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/

安装完成之后，使用：
```python
from selenium import webdriver
driver=webdriver.Firefox()
## 做一些事情
driver.close()
```

就可以打开网络驱动器，此时我们可以看到一个Firefox窗口被打开。同样注意的是，用完之后记得关掉。

比如，如果使用之前的方法获取京东的价格：

In [3]:
import requests
r=requests.get("https://item.jd.com/100008550055.html",headers=headers)
bs = BeautifulSoup(r.text, "html.parser")
print(bs.find(name='span',attrs={'class':'p-price'}))

<span class="p-price">
<span>￥</span>
<span class="price J-p-100008550055"></span>
</span>


可以看到价格是空缺的，其原因是因为这个价格是动态加载的，在最开始的时候的HTML并没有价格信息，而是后续的JavaScript中获得的。

为了解决这个问题，第一种办法是追踪这个页面的所有JavaScript，或者由JavaScript发送的请求，逐步分析到底是哪一个请求获取的价格，然后发送一个相同的请求。

这种方法的好处是程序运行快，但是需要掌握比较全面的JavaScript、HTTP协议等相关知识。

一个比较简单的方法是使用selenium直接打开一个浏览器，让浏览器自己去跑所有的JavaScript，拿到最终的HTML即可：

In [4]:
from selenium import webdriver
import time
import re
import json
from bs4 import BeautifulSoup

url = "https://item.jd.com/100008550055.html"
# 如果不需要Firefox窗口打开：
# from selenium.webdriver import FirefoxOptions
# opts = FirefoxOptions()
# opts.add_argument("--headless")
# driver = webdriver.Firefox(options=opts)

# 如果需要Firefox窗口打开，直接：driver=webdriver.Firefox()
driver = webdriver.Firefox()
driver.get(url)
html = driver.page_source
driver.close()
## 产品名
bs = BeautifulSoup(html, "html.parser")
print(bs.find(name='span', attrs={'class': 'p-price'}))


<span class="p-price">
<span>￥</span>
<span class="price J-p-100008550055">13999.00</span>
</span>


可以看到经过这样的步骤后，价格就可以被显示出来了。

selenium另外一个更常用的功能是自动填充、点击。为了实现这一目的，首先需要能够找到相应的按钮、输入框等，有如下方法可以使用：

* find_element_by_name
* find_element_by_id
* find_element_by_xpath
* find_element_by_link_text
* find_element_by_partial_link_text
* find_element_by_tag_name
* find_element_by_class_name
* find_element_by_css_selector

理解以上函数需要一些HTML、CSS、JavaScript的背景知识，我们在此不再详细讨论。在此，展示一个简单的例子：

In [5]:
from selenium import webdriver
import time
import re
import json
from bs4 import BeautifulSoup

url = "http://weixin.sogou.com"
# 如果不需要Firefox窗口打开：
from selenium.webdriver import FirefoxOptions

opts = FirefoxOptions()
opts.add_argument("--headless")
driver = webdriver.Firefox(options=opts)

# 如果需要Firefox窗口打开，直接：driver=webdriver.Firefox()
driver.get(url)
driver.find_element_by_id('query').send_keys("Python爬虫")  #模仿填写搜索内容
driver.find_element_by_class_name("swz").click()  #模仿点击搜索按钮
html = driver.page_source
driver.close()
bs = BeautifulSoup(html, "html.parser")
iterms = bs.find_all(name='h3')
for it in iterms:
    print(it.a.text, it.a['href'])


/tmp/ipykernel_1285/1824427840.py:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('query').send_keys("Python爬虫")  #模仿填写搜索内容
/tmp/ipykernel_1285/1824427840.py:18: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name("swz").click()  #模仿点击搜索按钮


Python爬虫有什么用,网友纷纷给出自己的答案,爬虫能做的还是很多的 /link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6cwJThYulHEtVjXrGTiVgS1FrTkI-DovXD9DcHczLf6pnbON0rLdqtVqXa8Fplpd9gQrNXM3slJJrt7vc5PIypa0lU1sgX8JSCKTdGK-Y2oZGEivLg_VOQTPTYbpdcNpOaIUOTItQpVBjpTCE9XWC352V1n9As4R9Dc3RDYCgTXT_NOewxp24wAhnK6mjA7NRWSyNRajz3kxI_voil5GJR4nqIuYyF4Gn0D7A00WMcTJr1dbZWbKUeQ..&type=2&query=Python%E7%88%AC%E8%99%AB&token=A5186204A5882D2490954B72B26DE761906D5E27624DA4AC
Python爬虫:一些常用的爬虫技巧总结 /link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6ft3wfAVofsP5Peu-UiA4DNAVN4b2JmnIr74MNvihD8WVgrFeYQugDuYzQu2p2pImwtoyrgGlkVlEdYjGlPy_qYXwZP3TSWF2zR0NC3MZDul4oV9veERHc_NufH6nIdqsPiZuCYrtAxmcwkYEdEk1J4e_2zg-AOldk2IrNem8ZOivAwmOPmb8KIQAGwM1OuDHjFnrIL-nb-a2U5V6TzTspgO11orYc6Sufa3B6vOUG-WqEaetqej1kp6rq1Hotv9rL7RGnVFHfwjYquOweQiR5fvf9LuPMwg0iZqwg0yoRFw.&type=2&query=Python%E7%88%AC%E8%99%AB&token=A5186204A5882D2490954B72B26DE761906D5E27624DA4AC
Python爬虫超详细讲解(零基础入门,小朋友都看的懂) /link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6cwJThYulHEtVjXrGTiVgS1FrTkI-DovXD9DcHczLf6pnbON0rLdq

## playwright的使用

另外一个值得使用的是微软刚刚推出的playwright(https://github.com/microsoft/playwright-python)。综合起来看，playwright比selenium可能更好用，比如playwright支持录制浏览器的操作等，可以很大程度上提高工作效率。

为了安装playwright，可以直接使用：
```sh
pip3 install playwright
playwright install
```
以上命令会自动将环境配置好。playwright支持Firefox、Webkit、Chromium三种内核，常见的浏览器几乎都支持。

比如上面的查看京东价格的代码，用playwright可以写成：
```python
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup

with sync_playwright() as p:
    browser = p.firefox.launch(headless=False)
    page = browser.new_page()
    page.goto("https://item.jd.com/100008550055.html",
              wait_until='networkidle')
    page.screenshot(path="example.png")
    html = page.content()
    browser.close()
bs = BeautifulSoup(html, "html.parser")
print(bs.find(name='span', attrs={'class': 'p-price'}))
```

playwright另一个强大的功能是通过我们自己的操作自动生成代码。直接在命令行（不是python中，在terminal里）输入：
```sh
playwright codegen weixin.sogou.com
```
就会打开一个界面，在这个界面里面做各种操作都会被记录为代码。这样我们就可以不用自己去分析网页的结构，直接使用录制的代码即可。

# 代理的使用

反爬的一种经常性手段是使用IP地址进行反爬。如果一个IP地址频繁、大量的访问，对方服务器可能会封禁这个IP。解决这个问题的办法是经常更换IP。然而，对于大多数个人电脑而言，更换IP是非常困难的，此时我们就需要使用代理了。

所谓“代理”，即我们不将请求直接发送给目标服务器，而是先发送给代理服务器，代理服务器代为将请求发送给目标服务器，这样目标服务器看到的IP地址就是代理服务器的IP地址而不是我们自己的IP地址了。

在requests中使用代理很简单，前面已经讲过：
```python
proxies = {
  'http': 'http://user@password@10.10.1.10:3128',
  'https': 'http://10.10.1.10:1080',
}
r=requests.get(url, proxies=proxies)
```
然而真正的问题是：**代理服务器从何而来？**

一般而言，我们需要自行搭建或者寻找代理服务器。

比如，我们可以使用阿里云、腾讯云、华为云、AWS、Azure等租赁一个简单的云服务器，然后将其配置为一个代理服务器。如果使用Linux，可以使用squid搭建代理服务器，如果使用Windows server，可以直接配置为代理服务器。

然而，如果租赁云服务器，一般云服务器会给一个固定的IP地址，也是没有办法很方便的更换IP地址的。一种简单的办法是租赁好多个云服务器，然后隔一段时间换一个代理，或者每次发送请求时随机选择一个代理即可。

不过这样做的缺点非常明显：成本非常高！

这个时候，我们提供两个策略：

1. 使用别人的免费代理，构建一个代理池
2. 使用带有ADSL拨号功能的服务器，每一次拨号就会更换一个IP地址

第一个策略需要一些动手能力。有一些开源项目已经可以给出了解决方案，比如这个项目：
> https://github.com/SpiderClub/haipproxy
不过仍然需要在使用时配置Redis等。

另一个办法，通过ADSL拨号，可以找到带有ADSL拨号的服务器提供商，比如我使用的是云立方：
> http://www.yunlifang.cn
在该服务器上，每一次断开连接再pppoe拨号，那么IP地址就换掉了。

然而一个现实的问题是，如果这个有ADLS重新拨号的服务器IP地址变掉了，那么我们也就无从找到这个服务器，也就无法做代理。

解决这个问题的最简单的办法是在每次拨号之后，向一个有固定IP地址的服务器报备最新的IP地址，然后在使用代理时，先向这个固定IP的服务器查询最新的代理服务器IP，然后再使用这个可变IP地址服务器上的代理功能。

比如，在具有ADSL拨号功能的服务器，我们可以使用如下shell脚本：

```sh
#!/bin/sh
pppoe-connect &
sleep 120
pppoe-start
while true
do
    pppoe-stop
    sleep 1
    pppoe-start
    sleep 2
    dial_delta=1800
    ipaddr=`pppoe-status | grep 'inet ' | gawk '{print $2}'`
    wget "http://$FIX_IP:$FIX_PORT/setip?token=$TOKEN&next_dial=$dial_delta&ip=$ipaddr" -O setip.txt --timeout=2 --tries=1 -q
    status=`cat setip.txt`
    ipaddr=\"$ipaddr\"
    if [ $ipaddr = $status ]
    then
        echo $ipaddr
        sleep $dial_delta
    else
        echo "重新拨号......"
        continue
    fi
done
```
定时进行重新拨号，并报送远程IP地址，其中$FIX_IP为固定IP地址服务器的IP地址，而$FIX_PORT为固定IP服务器开放的接收端口，$TOKEN为两个服务器之间通信的一个“密码”，防止其他人恶意篡改。pppoe-start命令每执行一次，就会更换一次IP地址。

而在固定IP服务器，可以使用如下Python脚本接收更新的地址：
```python
#!/usr/bin/python3

from fastapi import FastAPI
import time
import uvicorn

app = FastAPI()

IP = '127.0.0.1'
last_time = time.time()
dial_delta = 1800


@app.get("/ip")
async def get_IP(token: str):
    if token == "dasfdsafdasfdjldasfkj":
        time_left = dial_delta - (time.time() - last_time)
        if time_left > -10:
            return {
                'ip': IP,
                'time_left': dial_delta - (time.time() - last_time)
            }
        else:
            return {'ip': '127.0.0.1', 'time_left': -1000}
    else:
        return "error"


@app.get("/setip")
async def set_IP(ip: str, token: str, next_dial: int = 1800):
    """
    ip: 要设置的IP
    token: 密码
    next_dial: 距离下次拨号的时间，默认为600秒，即10分钟
    """
    global IP
    global last_time
    global dial_delta
    if token == 'al;sdjkfljklasdjflkaj;lkf':
        IP = ip
        last_time = time.time()
        dial_delta = next_dial
        return IP
    else:
        return "error"


if __name__ == "__main__":
    uvicorn.run(app, host='0.0.0.0', port=9999)

```
以上就是一个简单的web服务器，/ip页面用于获取当前IP地址和下次更新还剩余的时间；/setip页面用于设置当前的ip地址。

然后，当我们需要获取当前的代理服务器地址时，只需要使用如下函数即可：
```python
def get_proxy_ip(min_available_time=10, retry=2):
    url = f"http://{FIX_IP}:{FIX_PORT}/ip?token={TOKEN}"
    return_ip = None
    for tt in range(retry):
        r = requests.get(url)
        if r.status_code == 200:
            ip_info = r.json()
            if ip_info['time_left'] < min_available_time:
                time.sleep(20)
            else:
                return_ip = ip_info['ip']
                return_ip = {
                    'http': return_ip + ':9998',
                    'https': return_ip + ':9998'
                }
                break
    return return_ip
```

# 后记：关于爬虫的一些补充

关于爬虫，这里有一些需要补充的话。

首先是关于爬虫的道德问题。实际上爬虫一直处在一个灰色地带：违法或者不违法，道德或者不道德。在此一些原则希望与大家分享：

* 按照我国法律的相关规定，如果网页内容、API接口使用了加密，破解加密是违法行为。
* 如果将爬取的数据商用，法律风险非常大
* 如果网站提供了接口（比如豆瓣API），不使用接口而直接爬取网页也是不道德的。
* 尽量爬取的速率不要太高，不要给他人的服务器造成太大负担。

最后，爬虫和反爬一直是相互伴生的两个技术，我们这里提供一些常用的反爬思路：

* 如果request到的内容与自己使用浏览器看到的内容不符：
    - 首先查看请求头部，最极端情况下，完全复制浏览器的请求头部，看看能不能得到相同的反应。
    - 是否需要cookies登录？
    - 是否有重定向？
    - 仔细查看request得到的html，里面可能有玄机
    - 实在不行，selenium
* 如果request中的内容没有自己想要的信息：
    - 是否是动态加载的？
        * 对比request得到的HTML和浏览器的HTML是否不同，如果是不同的，可能时动态加载的
        * 仔细分析JavaScript，或者浏览器发送的每一次请求
        * 尝试直接发送动态请求
    - 内容是否是加密的？
        * 仔细分析加密方法
* 爬取几个页面后被禁止访问：
    - 可能爬取的频率太频繁
    - 间隔时间小一点
    - 使用代理池（比如该项目： https://github.com/SpiderClub/haipproxy ）
    - 使用ADSL
* 需要登录：
    - 不频繁的登录通常使用cookies，模拟一次登录之后获得cookies即可
    - 频繁的登录或者验证码，可能需要结合图像识别之类的方法
    - 一个简单的登录模拟：https://github.com/CharlesPikachu/DecryptLogin

# 一个综合例子：爬取MBA智库的名词作为词典

我们接下来在进行文本分析时，一个重要的步骤是分词，也就是把句子分成一个个的词。然而经济、金融以及各个行业有很多专有名词，所以我们的想法是希望能够找到一个“词典”，里面包含了经济金融的常用词汇，这样我们在进行文本分析时就可以更可靠的区分出句子中的每个词。

我们想到的办法是从“MBA智库”中把所有的名词都爬取下来。

为此，我们首先定义一些后面遇到的函数，包括写错误日志的函数以及去除文本稳健重复行的函数：

```python
"""
经济词典中所使用的通用函数。包括：

errlog：错误日志
getdic：从github上下载已经整理好的数据文件
"""


def errlog(message):
    """
    写入错误日志到logfile.txt
    需要的输入：错误信息message
    """
    import os
    import time
    with open(
            os.path.dirname(os.path.realpath(__file__)) + "/logfile.txt",
            'a') as f:
        f.write(
            time.strftime("%Y-%m-%d %H:%M:%S  ", time.localtime()) + message +
            '\n')


def getdic(filename):
    """
    从GitHub上下载已经整理好的词典文件。
    需要的输入：文件名filename
    """
    import os
    import requests
    import sys
    url = "https://raw.githubusercontent.com/sijichun/jingjidic/master/sub_dics/" + filename
    PWD = os.path.dirname(os.path.realpath(__file__))
    try:
        html = requests.get(url).text
        with open(PWD + '/sub_dics/' + filename, 'w') as f:
            f.write(html)
    except Exception as e:
        print("错误：" + str(e))
        sys.exit(1)


def remove_duplicates(filename, csv=False):
    """
    去除文件filename中重复的行。
    """
    with open(filename, 'r') as f:
        if csv:
            import csv
            rows = []
            csv_file = csv.DictReader(f)
            for r in csv_file:
                rows.append(r)
            rows = list(set(rows))
        else:
            content = f.readlines()
            content = list(set(content))
            content.sort()
    with open(filename, 'w') as f:
        if csv:
            csv_file = csv.DictWriter(f, list(rows[0].keys()))
            csv_file.writeheader()
            csv_file.writerows(rows)
        else:
            for w in content:
                f.write(w.strip() + '\n')
```

接下来是爬取的过程：
```python
#!/usr/bin/python3

import os
import sys
import requests
from bs4 import BeautifulSoup
import time
import random
from functions import errlog
from functions import remove_duplicates
import csv
from queue import Queue

glossary = []
tasks = Queue()
Iterm_sets = set()

## get the current path
PWD = os.path.dirname(os.path.realpath(__file__))
SUB_PATH = PWD + '/sub_dics'
Dic_File = SUB_PATH + "/glossary.txt"
Csv_File = SUB_PATH + "/glossary.csv"
if os.path.exists(SUB_PATH) is not True:
    os.mkdir(SUB_PATH)


def get_glossary_sub_page(retry=3):
    """
    从Queue中挨个取出元素并爬取
    """
    URL = "https://wiki.mbalib.com%s"
    while not tasks.empty():
        father_iterm = tasks.get()
        for i in range(retry):
            try:
                if "https://" not in father_iterm['link']:
                    url = URL % father_iterm['link']
                html = requests.get(url, timeout=10).text
                # print(html)
                bs = BeautifulSoup(html, "html.parser")
                ## 获取所有子类
                links = bs.find_all(name="a")
                for l in links:
                    href = l.get('href')
                    if href != None and href.find(
                            "Category:") > 0 and l.text[-2:] not in ('标志',
                                                                     '图像'):
                        if l.text not in Iterm_sets:
                            iterm = {
                                'iterm': l.text,
                                'link': href,
                                'father': father_iterm['iterm'],
                                'tag': 'Category'
                            }
                            glossary.append(iterm)
                            tasks.put(iterm)
                            Iterm_sets.add(l.text)
                            print(iterm)
                ## 获取所有词语列表
                div = bs.find_all(name="div", attrs={"class": "page_ul"})[0]
                links = div.find_all(name='a')
                for l in links:
                    href = l.get('href')
                    if href != None and href.find("/wiki/") == 0:
                        if l.text not in Iterm_sets:
                            iterm = {
                                'iterm': l.text,
                                'link': href,
                                'father': father_iterm['iterm'],
                                'tag': 'Iterm'
                            }
                            glossary.append(iterm)
                            Iterm_sets.add(l.text)
                            with open(Dic_File, 'a') as f:
                                f.write(l.text + '\n')
                            print(iterm)
                break
            except Exception as e:
                print("glossary错误：" + str(e) + "：" + url +
                      "：函数get_glossary_sub_page")
                errlog("glossary错误：" + str(e) + "：" + url +
                       "：函数get_glossary_sub_page")
                os.system("pppoe-stop; pppoe-start")
                time.sleep(random.random() * 10 + i * 10)
        time.sleep(3 + 10 * random.random())
    return


def get_glossary(retry=3):
    """
    从MBA智库百科中爬取专业词汇。
    """
    URL = "https://wiki.mbalib.com%s"
    url = URL % "/wiki/MBA智库百科:分类索引"
    for i in range(retry):
        try:
            html = requests.get(url, timeout=10).text
            bs = BeautifulSoup(html, "html.parser")
            links = bs.find_all(name="a")
            for l in links:
                href = l.get('href')
                if href != None and href.find(
                        "Category:") > 0 and l.text[-2:] not in ('标志', '图像'):
                    if l.text not in Iterm_sets:
                        iterm = {
                            'iterm': l.text,
                            'link': href,
                            'father': 'ROOT',
                            'tag': 'Category'
                        }
                        glossary.append(iterm)
                        tasks.put(iterm)
                        Iterm_sets.add(l.text)
                        with open(Dic_File, 'a') as f:
                            f.write(l.text + '\n')
                        print(iterm)
            break
        except Exception as e:
            print("glossary错误：" + str(e) + "：" + url + "：函数get_glossary")
            errlog("glossary错误：" + str(e) + "：" + url + "：函数get_glossary")
            os.system("pppoe-stop; pppoe-start")
            time.sleep(random.random() * 10 + i * 10)
    ## 开始爬取子页面
    get_glossary_sub_page(retry=retry)
    return


if __name__ == "__main__":
    ## 获取命令行参数
    Need_Craw = False
    args = sys.argv[1:]
    for arg in args:
        if arg == "-update":
            Need_Craw = True
        else:
            print("无效的参数：" + arg)
            sys.exit(1)

    if Need_Craw:
        ## 从网络爬取
        get_glossary()
        ## 写入文件
        remove_duplicates(Dic_File)
        ## 写入csv文件
        csv_exist = os.path.exists(Csv_File)
        with open(Csv_File, 'a') as f:
            header = ['iterm', 'link', 'father', 'tag']
            f_csv = csv.DictWriter(f, header)
            if not csv_exist:
                f_csv.writeheader()
            f_csv.writerows(glossary)
    else:
        from functions import getdic
        getdic("glossary.txt")
        print("Complete. See ./sub_dics/glossary.txt")
```

注意我们使用了**队列**（**Queue**）来组织我们的爬取任务，每次得到需要爬取的页面就放在队列里面，然后不断的从队列里面取出任务再爬取子页面。

除此之外，我们这里租用了VPS服务器进行爬取，该服务器可以使用：
```shell
pppoe-stop
pppoe-start
```
的方式重新进行PPPoE拨号，每次拨号IP地址都会变化，从而避免了IP被封。